In [2]:
import os
from pathlib import Path

def find_project_root(max_levels=5):
    """
    Encontra o diretório raiz do projeto procurando pela pasta .venv
    
    Args:
        max_levels (int): Número máximo de níveis para subir na hierarquia
    
    Returns:
        Path: Caminho do diretório raiz do projeto
    
    Raises:
        RuntimeError: Se não encontrar o diretório raiz dentro do limite de níveis
    """
    current = Path.cwd()
    for _ in range(max_levels):
        if (current / '.venv').exists():
            return current
        parent = current.parent
        if parent == current:  # Chegou na raiz do sistema
            break
        current = parent
    
    raise RuntimeError(
        f'Não foi possível encontrar o diretório raiz do projeto em {max_levels} níveis acima.'
    )

# Configura o diretório de trabalho para a raiz do projeto
project_root = find_project_root()
os.chdir(project_root)
print(f'Diretório de trabalho configurado para: {project_root}')

Diretório de trabalho configurado para: g:\py_projects\investment_analyzer_api


In [ ]:
# Consultando o banco de dados Investment Manager

# Neste notebook, vamos nos conectar ao banco de dados SQLite "investment_manager.db" e realizar consultas para análise dos dados.

In [5]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados
database = 'app/data/database/investment_manager.db'
conn = sqlite3.connect(database)
cursor = conn.cursor()

# Verificar as tabelas disponíveis no banco de dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tabelas disponíveis no banco de dados:")
for table in tables:
    print(f"- {table[0]}")

Tabelas disponíveis no banco de dados:
- status
- country
- transaction_types
- users
- market
- assets
- portfolio
- asset_price_history
- asset_transactions
- alembic_version


In [5]:
# Obter o preço historico por ativo assim como o yf.donload
import pandas as pd
df_acoes_b3 = pd.read_csv("app/data/acoes-listadas-b3.csv")
df_acoes_b3["Ticker"] = df_acoes_b3["Ticker"].apply(lambda x: f"{x}.SA")
# set(df_acoes_b3["Ticker"])


df_acoes_b3.to_csv("app/data/list_assets.csv")

In [ ]:
import sqlite3

# Caminho do banco
DB_PATH = "app/data/database/investment_manager.db"

# Conecta ao banco
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# 1️⃣ Busca o ID do ativo pelo símbolo
def asset_history(symbol, conn):
    cursor.execute("SELECT id FROM assets WHERE symbol = ?", (symbol,))
    row = cursor.fetchone()
    if row:
        asset_id = row[0]
        print(f"ID do ativo: {asset_id}")
        # 2️⃣ Busca histórico de preços usando o ID
        cursor.execute("SELECT * FROM asset_price_history WHERE asset_id = ?", (asset_id,))
        price_history = cursor.fetchall()
        for line in price_history:
            print(line)
    else:
        print("Ativo não encontrado.")
    conn.close()

symbol = "SAPR4.SA"  # exemplo

asset_history(symbol, conn=conn)

ID do ativo: 4
(64, '2025-09-29 00:00:00.000000', 7.150000095367432, 7.269999980926514, 7.110000133514404, 7.210000038146973, 942800, 0.0, 0.0, 4)
(65, '2025-09-30 00:00:00.000000', 7.230000019073486, 7.28000020980835, 7.139999866485596, 7.25, 1094900, 0.0, 0.0, 4)
(66, '2025-10-01 00:00:00.000000', 7.239999771118164, 7.25, 7.110000133514404, 7.130000114440918, 788300, 0.0, 0.0, 4)
(67, '2025-10-02 00:00:00.000000', 7.130000114440918, 7.139999866485596, 6.949999809265137, 7.010000228881836, 1182700, 0.0, 0.0, 4)
(68, '2025-10-03 00:00:00.000000', 6.980000019073486, 7.059999942779541, 6.980000019073486, 7.010000228881836, 1057800, 0.0, 0.0, 4)
(69, '2025-10-06 00:00:00.000000', 6.989999771118164, 7.110000133514404, 6.940000057220459, 7.070000171661377, 1227600, 0.0, 0.0, 4)
(70, '2025-10-07 00:00:00.000000', 7.070000171661377, 7.070000171661377, 6.800000190734863, 6.809999942779541, 990400, 0.0, 0.0, 4)
(71, '2025-10-08 00:00:00.000000', 6.820000171661377, 6.869999885559082, 6.719999790

In [ ]:
symbol = "SAPR4.SA"

import sqlite3

# Conecta (ou cria se não existir)
conn = sqlite3.connect(database)

# Cria um cursor para executar comandos SQL
cur = conn.cursor()

# Cria uma tabela
cur.execute("""
CREATE TABLE IF NOT EXISTS usuarios (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    idade INTEGER
)
""")

# Insere dados
cur.execute("INSERT INTO usuarios (nome, idade) VALUES (?, ?)", ("Guilherme", 26))
conn.commit()

# Consulta os dados
cur.execute("SELECT * FROM usuarios")
for linha in cur.fetchall():
    print(linha)

# Fecha a conexão
conn.close()


OperationalError: near ".": syntax error

In [ ]:
def get_table_info(table_name):
    """Retorna informações sobre as colunas de uma tabela específica."""
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    
    print(f"\nInformações da tabela '{table_name}':")
    for col in columns:
        print(f"- {col[1]} ({col[2]})")

def query_to_dataframe(query):
    """Executa uma consulta SQL e retorna os resultados em um DataFrame."""
    return pd.read_sql_query(query, conn)

# Exemplo de uso: escolha uma tabela da lista anterior e visualize sua estrutura
if tables:
    get_table_info(tables[0][0])